In [ ]:
import sys
import serial
import time
import csv
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout,
    QPushButton, QLabel, QMenu, QAction, QComboBox,
    QTextEdit, QFileDialog, QTimeEdit, QWidgetAction,
    QGroupBox
)
from PyQt5.QtCore import QTimer, QTime, Qt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import matplotlib.pyplot as plt

class MultiSensorPlot(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Gas Chromatography UI")
        self.setGeometry(100, 100, 1280, 800)

        self.port = 'COM3'
        self.baudrate = 115200
        self.ser = None
        self.time_data = []
        self.tcd_data = []
        self.tvoc_data = []
        self.start_time = None
        self.auto_stop_enabled = False
        self.end_time = 0
        
        main_layout = QVBoxLayout(self)
        plot_layout = self.create_plot_layout()
        main_layout.addLayout(plot_layout)
        controls_layout = self.create_controls_layout()
        main_layout.addLayout(controls_layout)
        
        self.setup_axes_labels()

        self.connect_serial()
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_plot)
        self.timer.start(100)

    def create_plot_layout(self):
        main_plot_layout = QHBoxLayout()
        left_plots_layout = QVBoxLayout()
        
        self.fig1, self.ax1 = plt.subplots()
        self.canvas1 = FigureCanvas(self.fig1)
        
        self.fig3, self.ax3 = plt.subplots()
        self.canvas3 = FigureCanvas(self.fig3)
        
        self.line1, = self.ax1.plot([], [], color='red')
        self.line3, = self.ax3.plot([], [], color='blue')
        
        left_plots_layout.addWidget(self.canvas1)
        left_plots_layout.addWidget(self.canvas3)
        
        self.fig2, self.ax2 = plt.subplots()
        self.canvas2 = FigureCanvas(self.fig2)
        self.ax2_twin = self.ax2.twinx()
        
        self.line2, = self.ax2.plot([], [], color='red')
        self.line2_twin, = self.ax2_twin.plot([], [], color='blue')
        
        main_plot_layout.addLayout(left_plots_layout, 2)
        main_plot_layout.addWidget(self.canvas2, 3)

        return main_plot_layout

    def create_controls_layout(self):
        bottom_layout = QHBoxLayout()
        left_column_layout = QVBoxLayout()
        
        button_style = """
            QPushButton {
                background-color: #E0E0E0; border: 1px solid #B0B0B0;
                border-radius: 15px; padding: 10px; font-size: 14px;
            }
            QPushButton:hover { background-color: #D0D0D0; }
            QPushButton:pressed { background-color: #C0C0C0; }
        """

        main_buttons_layout = QHBoxLayout()
        btn_start = QPushButton("Start"); btn_start.clicked.connect(self.start_reading)
        btn_stop = QPushButton("Stop"); btn_stop.clicked.connect(self.stop_reading)
        
        btn_calibrate = QPushButton("Calibrate"); btn_calibrate.clicked.connect(self.calibrate_action)
        
        self.menu_button = QPushButton("Menu"); self.create_menu()
        
        for btn in [btn_start, btn_stop, btn_calibrate, self.menu_button]:
            main_buttons_layout.addWidget(btn)
            btn.setStyleSheet(button_style)

        pump_buttons_layout = QHBoxLayout()
        btn_pump_on = QPushButton("Pump ON"); btn_pump_on.clicked.connect(lambda: self.send_command(b'4'))
        btn_pump_off = QPushButton("Pump OFF"); btn_pump_off.clicked.connect(lambda: self.send_command(b'5'))
        
        pump_buttons_layout.addStretch()
        pump_buttons_layout.addWidget(btn_pump_on)
        pump_buttons_layout.addWidget(btn_pump_off)
        pump_buttons_layout.addStretch()
        for btn in [btn_pump_on, btn_pump_off]:
            btn.setStyleSheet(button_style)

        timer_layout = QVBoxLayout()
        timer_label = QLabel("Set Timer"); timer_label.setAlignment(Qt.AlignCenter)
        self.time_edit = QTimeEdit(); self.time_edit.setDisplayFormat("HH:mm:ss")
        self.time_edit.setTime(QTime(0, 0, 0)); self.time_edit.setStyleSheet("font-size: 14px; padding: 4px;")
        self.time_edit.setMaximumWidth(140)
        timer_layout.addWidget(timer_label); timer_layout.addWidget(self.time_edit)

        left_column_layout.addLayout(main_buttons_layout)
        left_column_layout.addLayout(pump_buttons_layout)
        left_column_layout.addStretch()

        timer_wrapper_layout = QHBoxLayout()
        timer_wrapper_layout.addStretch()
        timer_wrapper_layout.addLayout(timer_layout)
        timer_wrapper_layout.addStretch()
        left_column_layout.addLayout(timer_wrapper_layout)
        
        bottom_layout.addLayout(left_column_layout)

        self.log_box = QTextEdit(); self.log_box.setReadOnly(True)
        group_monitor = QGroupBox("Serial Monitor")
        monitor_layout = QVBoxLayout(); monitor_layout.addWidget(self.log_box)
        group_monitor.setLayout(monitor_layout)
        bottom_layout.addWidget(group_monitor)

        return bottom_layout

    def create_menu(self):
        self.menu = QMenu()
        save_action = QAction('Save Data', self); save_action.triggered.connect(self.save_data)
        self.menu.addAction(save_action); self.menu.addSeparator()

        com_label = QLabel("Select COM Port:"); com_label.setContentsMargins(5, 0, 5, 0)
        com_action_label = QWidgetAction(self.menu); com_action_label.setDefaultWidget(com_label)
        self.menu.addAction(com_action_label)

        com_selector = QComboBox(); com_selector.addItems([f'COM{i}' for i in range(1, 11)])
        com_selector.setCurrentText(self.port); com_selector.currentTextChanged.connect(self.change_com_port)
        com_action = QWidgetAction(self.menu); com_action.setDefaultWidget(com_selector)
        self.menu.addAction(com_action)
        
        baud_label = QLabel("Select Baud Rate:"); baud_label.setContentsMargins(5, 5, 5, 0)
        baud_action_label = QWidgetAction(self.menu); baud_action_label.setDefaultWidget(baud_label)
        self.menu.addAction(baud_action_label)
        
        baud_selector = QComboBox(); baud_selector.addItems(['9600', '19200', '38400', '57600', '115200'])
        baud_selector.setCurrentText(str(self.baudrate)); baud_selector.currentTextChanged.connect(self.change_baudrate)
        baud_action = QWidgetAction(self.menu); baud_action.setDefaultWidget(baud_selector)
        self.menu.addAction(baud_action)
        
        self.menu_button.setMenu(self.menu)
        
    def calibrate_action(self):
        if self.start_time is None:
            self.log_box.append("Cannot calibrate. Please start a run first.")
            return

        self.send_command(b'2')
        self.log_box.append("Live calibration event triggered.")

        self.start_time = time.time()

        self.time_data.clear()
        self.tcd_data.clear()
        self.tvoc_data.clear()
        
        for line in [self.line1, self.line2, self.line2_twin, self.line3]:
            line.set_data([], [])
        
        for canvas in [self.canvas1, self.canvas2, self.canvas3]:
            canvas.draw()
            
        self.log_box.append("Retention time reset. Continuing measurement...")

    def connect_serial(self):
        try:
            if self.ser and self.ser.is_open: self.ser.close()
            self.ser = serial.Serial(self.port, self.baudrate, timeout=0.1); self.ser.flushInput()
            self.log_box.append(f"Connected to {self.port} at {self.baudrate} baud.")
        except Exception as e:
            self.log_box.append(f"Serial connection failed: {e}")

    def change_com_port(self, text):
        self.port = text; self.connect_serial()

    def change_baudrate(self, text):
        self.baudrate = int(text); self.connect_serial()

    def send_command(self, cmd: bytes):
        if self.ser and self.ser.is_open:
            self.ser.write(cmd)
            cmd_map = {b'0': 'Stop', b'1': 'Start', b'2': 'Calibrate', b'4': 'Pump ON', b'5': 'Pump OFF'}
            self.log_box.append(f"Command sent: {cmd_map.get(cmd, 'Unknown')}")

    def start_reading(self):
        if self.ser and self.ser.is_open:
            self.time_data.clear(); self.tcd_data.clear(); self.tvoc_data.clear()
            for line in [self.line1, self.line2, self.line2_twin, self.line3]:
                line.set_data([], [])
            for ax in [self.ax1, self.ax2, self.ax3, self.ax2_twin]:
                ax.relim(); ax.autoscale_view()
            for canvas in [self.canvas1, self.canvas2, self.canvas3]:
                canvas.draw()
            
            self.send_command(b'1')
            self.start_time = time.time()
            t = self.time_edit.time()
            self.end_time = t.hour()*3600 + t.minute()*60 + t.second()
            self.auto_stop_enabled = self.end_time > 0
            
            self.log_box.append("Started reading...")
            if self.auto_stop_enabled: self.log_box.append(f"Measurement will stop automatically in {t.toString('HH:mm:ss')}.")
        else:
            self.log_box.append("Cannot start. Serial is not connected.")

    def stop_reading(self):
        self.send_command(b'0'); self.auto_stop_enabled = False
        if self.start_time is not None: self.log_box.append("Stopped reading.")
        self.start_time = None

    def update_plot(self):
        if self.ser and self.ser.in_waiting > 0 and self.start_time is not None:
            try:
                line = self.ser.readline().decode('utf-8').strip()
                if not line: return
                self.log_box.append(f"Serial: {line}")
                
                if "TCD (mV):" in line and "TVOC (ppb):" in line:
                    parts = line.split('|'); tcd_str = parts[0].split(':')[1].strip()
                    tvoc_str = parts[1].split(':')[1].strip()
                    tcd = float(tcd_str); tvoc = float(tvoc_str)
                    
                    current_time = time.time() - self.start_time
                    self.time_data.append(current_time); self.tcd_data.append(tcd); self.tvoc_data.append(tvoc)

                    self.line1.set_data(self.time_data, self.tcd_data)
                    self.line2.set_data(self.time_data, self.tcd_data)
                    self.line2_twin.set_data(self.time_data, self.tvoc_data)
                    self.line3.set_data(self.time_data, self.tvoc_data)

                    for ax in [self.ax1, self.ax2, self.ax3]:
                        ax.relim()
                        ax.autoscale_view(scalex=True, scaley=False)

                    if self.tcd_data:
                        tcd_min, tcd_max = min(self.tcd_data), max(self.tcd_data)
                        padding = (tcd_max - tcd_min) * 0.1
                        if padding < 1: padding = 1
                        self.ax1.set_ylim(tcd_min - padding, tcd_max + padding)
                        self.ax2.set_ylim(tcd_min - padding, tcd_max + padding)
                    
                    self.ax3.relim(); self.ax3.autoscale_view(scalex=False, scaley=True)
                    self.ax2_twin.relim(); self.ax2_twin.autoscale_view(scalex=False, scaley=True)

                    for canvas in [self.canvas1, self.canvas2, self.canvas3]:
                        canvas.draw()

                    if self.auto_stop_enabled and (current_time >= self.end_time):
                        self.stop_reading(); self.save_data()
            except Exception as e:
                self.log_box.append(f"Error processing data: {e} | Line: '{line}'")

    def setup_axes_labels(self):
        x_label = "Retention Time (s)"
        
        self.ax1.set_title("TCD Chromatogram")
        self.ax1.set_xlabel(x_label)
        self.ax1.set_ylabel("Detector Response (mV)")
        self.ax1.grid(True)
        
        self.ax2.set_title("Combined Detector Response")
        self.ax2.set_xlabel(x_label)
        
        self.ax2.set_ylabel("Detector Response (mV)", color='red', rotation=90, labelpad=15)
        self.ax2.tick_params(axis='y', labelcolor='red')
        self.ax2.spines['left'].set_color('red')
        
        self.ax2_twin.set_ylabel("Concentration (ppb)", color='blue', rotation=270, labelpad=20) 
        self.ax2_twin.tick_params(axis='y', labelcolor='blue')
        self.ax2_twin.spines['right'].set_color('blue')
        self.ax2.grid(True)

        self.ax3.set_title("Gas Sensor Response")
        self.ax3.set_xlabel(x_label)
        self.ax3.set_ylabel("Concentration (ppb)")
        self.ax3.grid(True)
        
        try:
            self.fig1.subplots_adjust(left=0.18, bottom=0.18)
            self.fig3.subplots_adjust(left=0.18, bottom=0.18)
            self.fig2.tight_layout(pad=1.5)
        except Exception:
            pass

    def save_data(self):
        if not self.time_data: self.log_box.append("No data to save."); return
        filename, _ = QFileDialog.getSaveFileName(self, "Save Data", "", "CSV Files (*.csv)")
        if filename:
            try:
                with open(filename, 'w', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(["Time (s)", "TCD (mV)", "TVOC (ppb)"])
                    for t, v1, v2 in zip(self.time_data, self.tcd_data, self.tvoc_data):
                        writer.writerow([f"{t:.2f}", f"{v1:.4f}", f"{v2:.4f}"])
                self.log_box.append(f"Data saved to {filename}")
            except Exception as e:
                self.log_box.append(f"Failed to save data: {e}")

def main():
    app = QApplication(sys.argv)
    window = MultiSensorPlot()
    window.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()